In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [10]:
df1 = pd.DataFrame(np.random.randint(low = 1, high = 7, size = (100,2)), columns = ('Кость_1','Кость_2'))
df2 = pd.DataFrame(np.random.randint(low = 1, high = 7, size = (100,2)), columns = ('Кость_1','Кость_2'))
df3 = pd.DataFrame(np.random.randint(low = 1, high = 7, size = (1000,2)), columns = ('Кость_1','Кость_2'))
df1['Сумма'] = df1['Кость_1'] + df1['Кость_2']
df2['Сумма'] = df2['Кость_1'] + df2['Кость_2']
df3['Сумма'] = df3['Кость_1'] + df3['Кость_2']

## Bar

Столбчатые диаграммы можно сформировать и своими силами, если сгруппировать данные и вычислить высоты столбцов.

Далее, используя класс **go.Bar** передаём названия столбцов и их величины в 2 аргумента:

In [39]:
my_group = df3.head(10).groupby(['Сумма']).count()

labels = my_group.index
values = my_group['Кость_1'].values

fig = go.Figure(data = [go.Bar(x= labels, y = values)])
fig.show()

Создадим парную гистограмму для 2 наборов по 100 бросков

In [64]:
labels_1 = df1.groupby(['Сумма']).count().index
labels_2 = df2.groupby(['Сумма']).count().index
values_1 = df1.groupby(['Сумма']).count()['Кость_1'].values
values_2 = df2.groupby(['Сумма']).count()['Кость_1'].values

In [65]:
fig = go.Figure()
fig.add_trace(go.Bar(x = labels_1, y = values_1, name = 'df1'))
fig.add_trace(go.Bar(x = labels_1, y = values_2, name = 'df2'))
fig.update_layout(title = 'Плотность распределения результатов',
                 xaxis_title = 'Результат броска', 
                 yaxis_title = 'Плотность')
fig.show()

Зато есть у такой диаграммы и серьёзное преимущество — высота столбцов не обязана быть положительной. Это можно использовать, например, для построения диаграммы прибылей и убытков по периодам. Просто вычтем из одного набора значений другой, получим список чисел (некоторые отрицательные, некоторые положительные, а некоторые 0) и используем его для построения.

In [98]:
labels = tuple(range(2,13))

clear = df1.groupby(['Сумма']).count()['Кость_1'].append(pd.DataFrame([0]*len(labels), index=labels)).\
reset_index().drop_duplicates('index').sort_values('index')
values_1 = clear[0]

clear2 = df2.groupby(['Сумма'])['Кость_1'].count().append(pd.DataFrame([0]*len(labels), index = labels)).\
reset_index().drop_duplicates('index').sort_values('index')
values_2 = clear2[0]

In [104]:
fig = go.Figure()
fig.add_trace(go.Bar(x = labels , y = values_1 - values_2))
fig.update_yaxes(zeroline=True, zerolinewidth=3, zerolinecolor='red')
fig.update_layout(title = 'Разница исходов',
                 xaxis_title = 'исходы',
                 yaxis_title = 'разница в числе исходов',
                 margin = dict(r = 0,l = 0,t = 30, b = 0))
fig.show()

А ещё можно вывести подписи прямо поверх столбцов. Для этого пригодится пара аргументов:

   * text — сюда передаём список тех значений, которые надо вывести (можно заранее сформировать произвольные строки)
   * textposition — способ вывода текста:
        * 'auto' — Plotly самостоятельно пример решение
        * 'outside' — снаружи, в случае столбчатой диаграммы это будет над столбцом с положительной высотой и под столбцом с отрицательной высотой
        * 'inside' внутри прямоугольника (если высота прямоугольника = 0, то надпись не будет видно)
        и т.д.


In [105]:
fig = go.Figure()
fig.add_trace(go.Bar(x = labels , y = values_1 - values_2, text = values_1 - values_2,
                    textposition = 'outside'))
fig.update_yaxes(zeroline=True, zerolinewidth=3, zerolinecolor='red')
fig.update_layout(title = 'Разница исходов',
                 xaxis_title = 'исходы',
                 yaxis_title = 'разница в числе исходов',
                 margin = dict(r = 0,l = 0,t = 30, b = 0))
fig.show()

А что, если мы хотим вывести не вертикальные столбцы, а горизонтальные полоски?

Это легко сделать надо только:

   * Добавить аргумент orientation='h'
   * Поменять местами x и y в данных и подписях (а так же везде, где мы задаём подписи осей, осевые линии и т.п.)


In [108]:
fig = go.Figure()
fig.add_trace(go.Bar(x = values_1 - values_2 , y = labels , orientation = 'h', text = values_1 - values_2,
                    textposition = 'outside'))
fig.update_xaxes(zeroline=True, zerolinewidth=3, zerolinecolor='red')
fig.update_layout(title = 'Разница исходов',
                 xaxis_title = 'разница в числе исходов',
                 yaxis_title = 'исходы',
                 margin = dict(r = 0,l = 0,t = 30, b = 0))
fig.show()

Так же при отображении гистограмм 2 наборов данных есть полезный аргумент для слоя — barmode='stack'.

Он позволяет объединять столбцы в общие колонки. Это полезно, если мы представляем наши данные, как единую серию экспериментов, т.е. мы бросили 100 раз кубики, потом ещё 100 раз и хотим узнать что вышло в итоге, сколько всё-таки каких исходов.


In [115]:
fig = go.Figure()
fig.add_trace(go.Bar(x = labels, y = values_1, name = 'df1'))
fig.add_trace(go.Bar(x = labels, y = values_2, name = 'df2'))
fig.update_layout(barmode = 'stack')
fig.show()